Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(verbose=1)

train_dataset = np.reshape(train_dataset, (train_dataset.shape[0],784))
new_train_dataset = train_dataset[:1000,]
new_train_labels = train_labels[:1000,]

test_dataset = np.reshape(test_dataset, (test_dataset.shape[0],784))

print(new_train_dataset.shape)

print("Fitting logistic regression")
logisticRegr.fit(new_train_dataset, new_train_labels)

print("Scorring logistic regression")
score = logisticRegr.score(test_dataset, test_labels)
print("Accuracy: "+str(score*100))

In [ ]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 5000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))  + 0.006 * tf.nn.l2_loss(weights)

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 10000

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 1000 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


In [19]:
batch_size = 32

n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
learning_rate = 0.001


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_classes])
    
    weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    logits = tf.matmul(layer_2, weights['out']) + biases['out']

    regularization_term = 0
    for key, w in weights.items():
        regularization_term += tf.nn.l2_loss(w)
    regularization_term *= 0.04
    
    # Optimizer.
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)) + regularization_term
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    # Predictions for the training, validation, and test data.
    #train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    train_op = optimizer.minimize(loss_op)
    # Initializing the variables
    init = tf.global_variables_initializer()

In [20]:
num_steps = 1000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    avg_cost = 0
    avg_acc = 0
    
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    for step in range(1,num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {X : batch_data, Y : batch_labels}

        #l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
        _, c = session.run([train_op, loss_op], feed_dict={X: batch_data, Y: batch_labels})
        # Compute average loss
        avg_cost += c
        avg_acc += accuracy.eval({X: valid_dataset, Y: valid_labels})
        if step % 100 == 0:
            print("Epoch:", '%04d' % (step+1), "cost={:.9f}".format(avg_cost/step))
            print("Minibatch accuracy: %.1f%%" % (100*accuracy.eval({X: batch_data, Y: batch_labels})))
            print("Validation accuracy: %.1f%%" % (avg_acc*100/step))
            print("Test accuracy: %.1f%%" % (100*accuracy.eval({X: test_dataset, Y: test_labels})))
        #if (step % 1 == 0):
            #print("Minibatch loss at step %d: %f" % (step, l))
            #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
    

Initialized
Epoch: 0101 cost=6448.640971680
Minibatch accuracy: 87.5%
Validation accuracy: 61.0%
Test accuracy: 78.7%
Epoch: 0201 cost=6175.485322266
Minibatch accuracy: 81.2%
Validation accuracy: 66.0%
Test accuracy: 79.4%
Epoch: 0301 cost=6042.145146484
Minibatch accuracy: 81.2%
Validation accuracy: 67.9%
Test accuracy: 80.2%
Epoch: 0401 cost=5939.238605957
Minibatch accuracy: 90.6%
Validation accuracy: 69.3%
Test accuracy: 81.7%
Epoch: 0501 cost=5855.528302734
Minibatch accuracy: 93.8%
Validation accuracy: 70.1%
Test accuracy: 81.5%
Epoch: 0601 cost=5787.988625488
Minibatch accuracy: 78.1%
Validation accuracy: 70.6%
Test accuracy: 81.0%
Epoch: 0701 cost=5733.145507812
Minibatch accuracy: 90.6%
Validation accuracy: 71.0%
Test accuracy: 79.9%
Epoch: 0801 cost=5682.017655029
Minibatch accuracy: 78.1%
Validation accuracy: 71.2%
Test accuracy: 82.5%
Epoch: 0901 cost=5636.191954210
Minibatch accuracy: 90.6%
Validation accuracy: 71.4%
Test accuracy: 82.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [29]:
batch_size = 32

n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
learning_rate = 0.001


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)
    
    weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #dropout
    drop_out = tf.nn.dropout(layer_2, keep_prob) 
    # Output fully connected layer with a neuron for each class
    logits = tf.matmul(drop_out, weights['out']) + biases['out']

    regularization_term = 0
    for key, w in weights.items():
        regularization_term += tf.nn.l2_loss(w)
    regularization_term *= 0.04
    
    # Optimizer.
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)) + regularization_term
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    # Predictions for the training, validation, and test data.
    #train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    train_op = optimizer.minimize(loss_op)
    # Initializing the variables
    init = tf.global_variables_initializer()

num_steps = 501

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    avg_cost = 0
    avg_acc = 0
    
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    for step in range(1,num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {X : batch_data, Y : batch_labels}

        #l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
        _, c = session.run([train_op, loss_op], feed_dict={X: batch_data, Y: batch_labels, keep_prob:0.2})
        # Compute average loss
        avg_cost += c
        avg_acc += accuracy.eval({X: valid_dataset, Y: valid_labels, keep_prob:1.0})
        if step % 50 == 0:
            print("Epoch:", '%04d' % step, "cost={:.9f}".format(avg_cost/step))
            print("Minibatch accuracy: %.1f%%" % (100*accuracy.eval({X: batch_data, Y: batch_labels, keep_prob:1.0})))
            print("Validation accuracy: %.1f%%" % (avg_acc*100/step))
            print("Test accuracy: %.1f%%" % (100*accuracy.eval({X: test_dataset, Y: test_labels, keep_prob:1.0})))
        #if (step % 1 == 0):
            #print("Minibatch loss at step %d: %f" % (step, l))
            #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
    

Initialized
Epoch: 0050 cost=10454.474355469
Minibatch accuracy: 84.4%
Validation accuracy: 60.4%
Test accuracy: 80.3%
Epoch: 0100 cost=9198.384633789
Minibatch accuracy: 84.4%
Validation accuracy: 67.7%
Test accuracy: 83.4%
Epoch: 0150 cost=8655.408027344
Minibatch accuracy: 84.4%
Validation accuracy: 70.7%
Test accuracy: 84.8%
Epoch: 0200 cost=8298.084216309
Minibatch accuracy: 71.9%
Validation accuracy: 72.5%
Test accuracy: 85.1%
Epoch: 0250 cost=8072.241865234
Minibatch accuracy: 71.9%
Validation accuracy: 73.6%
Test accuracy: 85.9%
Epoch: 0300 cost=7852.641632487
Minibatch accuracy: 78.1%
Validation accuracy: 74.4%
Test accuracy: 86.2%
Epoch: 0350 cost=7673.316109096
Minibatch accuracy: 87.5%
Validation accuracy: 75.1%
Test accuracy: 86.2%
Epoch: 0400 cost=7513.564158936
Minibatch accuracy: 84.4%
Validation accuracy: 75.6%
Test accuracy: 86.7%
Epoch: 0450 cost=7371.286185981
Minibatch accuracy: 71.9%
Validation accuracy: 76.0%
Test accuracy: 86.1%
Epoch: 0500 cost=7243.164581055
M

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 32

n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
learning_rate = 0.001


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)
    
    weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #dropout
    drop_out = tf.nn.dropout(layer_2, keep_prob) 
    # Output fully connected layer with a neuron for each class
    logits = tf.matmul(drop_out, weights['out']) + biases['out']

    regularization_term = 0
    for key, w in weights.items():
        regularization_term += tf.nn.l2_loss(w)
    regularization_term *= 0.04
    
    # Optimizer.
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)) + regularization_term
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    # Predictions for the training, validation, and test data.
    #train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    train_op = optimizer.minimize(loss_op)
    # Initializing the variables
    init = tf.global_variables_initializer()

num_steps = 501

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    avg_cost = 0
    avg_acc = 0
    
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    for step in range(1,num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {X : batch_data, Y : batch_labels}

        #l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
        _, c = session.run([train_op, loss_op], feed_dict={X: batch_data, Y: batch_labels, keep_prob:0.2})
        # Compute average loss
        avg_cost += c
        avg_acc += accuracy.eval({X: valid_dataset, Y: valid_labels, keep_prob:1.0})
        if step % 50 == 0:
            print("Epoch:", '%04d' % step, "cost={:.9f}".format(avg_cost/step))
            print("Minibatch accuracy: %.1f%%" % (100*accuracy.eval({X: batch_data, Y: batch_labels, keep_prob:1.0})))
            print("Validation accuracy: %.1f%%" % (avg_acc*100/step))
            print("Test accuracy: %.1f%%" % (100*accuracy.eval({X: test_dataset, Y: test_labels, keep_prob:1.0})))
        #if (step % 1 == 0):
            #print("Minibatch loss at step %d: %f" % (step, l))
            #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
    

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [56]:
batch_size = 32

n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 128 # 2nd layer number of neurons
n_hidden_3 = 64 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
learning_rate = 0.001


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)
    
    global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    decay_learning_rate = tf.train.exponential_decay(learning_rate, global_step, 1000, 0.96)
    
    weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'b3': tf.Variable(tf.random_normal([n_hidden_3])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])

    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    #dropout
    drop_out = tf.nn.dropout(layer_3, keep_prob) 
    # Output fully connected layer with a neuron for each class
    logits = tf.matmul(drop_out, weights['out']) + biases['out']

    regularization_term = 0
    for key, w in weights.items():
        regularization_term += tf.nn.l2_loss(w)
    regularization_term *= 0.08
    
    
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    
    # Optimizer.
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)) + regularization_term
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=decay_learning_rate)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    # Predictions for the training, validation, and test data.
    #train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    train_op = optimizer.minimize(loss_op, global_step = global_step)
    # Initializing the variables
    init = tf.global_variables_initializer()

num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    avg_cost = 0
    avg_acc = 0
    
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    for step in range(1,num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {X : batch_data, Y : batch_labels}

        #l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)
        _, c = session.run([train_op, loss_op], feed_dict={X: batch_data, Y: batch_labels, keep_prob:0.5})
        # Compute average loss
        avg_cost += c
        avg_acc += accuracy.eval({X: valid_dataset, Y: valid_labels, keep_prob:1.0})
        if step % 100 == 0:
            print("Epoch:", '%04d' % step, "cost={:.9f}".format(avg_cost/step))
            print("Minibatch accuracy: %.1f%%" % (100*accuracy.eval({X: batch_data, Y: batch_labels, keep_prob:1.0})))
            print("Validation accuracy: %.1f%%" % (avg_acc*100/step))
            print("Test accuracy: %.1f%%" % (100*accuracy.eval({X: test_dataset, Y: test_labels, keep_prob:1.0})))
        #if (step % 1 == 0):
            #print("Minibatch loss at step %d: %f" % (step, l))
            #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
    

Initialized
Epoch: 0100 cost=21014.178544922
Minibatch accuracy: 78.1%
Validation accuracy: 62.5%
Test accuracy: 76.0%
Epoch: 0200 cost=17095.459526367
Minibatch accuracy: 78.1%
Validation accuracy: 65.8%
Test accuracy: 75.5%
Epoch: 0300 cost=15205.012903646
Minibatch accuracy: 87.5%
Validation accuracy: 67.5%
Test accuracy: 82.2%
Epoch: 0400 cost=14023.146362305
Minibatch accuracy: 87.5%
Validation accuracy: 68.6%
Test accuracy: 82.4%
Epoch: 0500 cost=13191.008332031
Minibatch accuracy: 93.8%
Validation accuracy: 69.3%
Test accuracy: 78.3%
Epoch: 0600 cost=12572.057727865
Minibatch accuracy: 78.1%
Validation accuracy: 69.8%
Test accuracy: 80.5%
Epoch: 0700 cost=12081.017367467
Minibatch accuracy: 87.5%
Validation accuracy: 70.2%
Test accuracy: 79.3%
Epoch: 0800 cost=11681.419104004
Minibatch accuracy: 81.2%
Validation accuracy: 70.5%
Test accuracy: 83.3%
Epoch: 0900 cost=11343.601323785
Minibatch accuracy: 90.6%
Validation accuracy: 70.8%
Test accuracy: 81.7%
Epoch: 1000 cost=11049.22

Epoch: 7900 cost=5805.698940275
Minibatch accuracy: 90.6%
Validation accuracy: 73.8%
Test accuracy: 83.6%
Epoch: 8000 cost=5771.477671600
Minibatch accuracy: 78.1%
Validation accuracy: 73.8%
Test accuracy: 80.0%
Epoch: 8100 cost=5737.665134127
Minibatch accuracy: 84.4%
Validation accuracy: 73.8%
Test accuracy: 82.9%
Epoch: 8200 cost=5704.254705275
Minibatch accuracy: 90.6%
Validation accuracy: 73.8%
Test accuracy: 81.1%
Epoch: 8300 cost=5671.234206661
Minibatch accuracy: 84.4%
Validation accuracy: 73.8%
Test accuracy: 81.2%
Epoch: 8400 cost=5638.595438843
Minibatch accuracy: 87.5%
Validation accuracy: 73.8%
Test accuracy: 81.8%
Epoch: 8500 cost=5606.336597570
Minibatch accuracy: 81.2%
Validation accuracy: 73.8%
Test accuracy: 81.5%
Epoch: 8600 cost=5574.445839390
Minibatch accuracy: 84.4%
Validation accuracy: 73.9%
Test accuracy: 84.0%
Epoch: 8700 cost=5542.914704646
Minibatch accuracy: 87.5%
Validation accuracy: 73.9%
Test accuracy: 82.2%
Epoch: 8800 cost=5511.743476535
Minibatch accu